# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [3]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [7]:
def expected_return(P, rf, mkt_rp):
    
    #rf + mkt_rp = exp_ret
    sam = P
    total = sam['Amount Invested'].sum()
    sam['weight'] = sam['Amount Invested'] / total
    sam['haha'] = ((sam['Beta'] * mkt_rp) + rf) * sam['weight']
    
    return sam['haha'].sum()

expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [39]:
def volatility(P, cor):
    
    sam = P
    leng = P.size
    total = sam['Amount Invested'].sum()
    sam['weight'] = sam['Amount Invested'] / total
    sam['wi'] = sam['weight'] * sam['Volatility'] # w_i * sig_i
    big = (sam['wi'].sum()) * cor
    sam['wi'] = ((sam['wi'] ** 2) * (1 - cor)) + (sam['wi'] * big)
    big = sam['wi'].sum()
    
    return big ** 0.5

volatility(portfolio, -0.2)

0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [40]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    return rf + ((vol / mkt_vol) * mkt_rp)

expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i - Rf, R_mkt – Rf) / Var(R_mkt - Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [2]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    
    df1 = pd.read_csv(return_path)
    df2 = pd.read_csv(factors_path)
    
    df1['date'] = (df1['year'] * 100) + df1['month']
    df2['RF'] = df2['RF'] / 100
    df2['Mkt-RF'] = df2['Mkt-RF'] / 100
    
    df1 = pd.merge(df1, df2, on='date')

    grouped = df1.groupby("CUSIP")
    grouped['a'] = df['RET'] - df1['RF']
    grouped['b'] = df1['Mkt-RF'] - df1['RF']
    beta2 = grouped['b'].var()
    beta1 = grouped.apply(lambda x: x['a'].cov(x['b']))
    betas = beta1 / beta2
    expected = grouped['RET'].mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : grouped.groups.keys(),
        "EXP_RETURN" : expected,
        "BETA" :  betas,
    })
    
    return output

return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data

NameError: name 'df' is not defined

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [133]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    
    return np.polyfit(data['BETA'], data['EXP_RETURN'], 1)[1]

slope(CAPM_beta('monthlycrsp_mod7.csv', "F-F_Research_Data_Factors.CSV"))

0.007947140194912806